Necessary imports

In [10]:
from EEGModels import EEGNet
import scipy
import numpy as np
from tensorflow.keras import utils as np_utils
from tensorflow.keras.callbacks import ModelCheckpoint
import os
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import balanced_accuracy_score


Extract data and split into 12 samples fortraining and 2 samples for validating the best model

In [11]:
folder_path = './PS_data_mat/'
file_list = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.mat')]

ok_P = True
ok_S = True

X = []
Y = []

for i, file in enumerate(file_list):
    x = scipy.io.loadmat(file)
    x = x['data']
    if ok_P and file[14] == 'P':
        ok_P = False
        X_validate_1 = x
        Y_validate_1 = np.zeros((x.shape[0],2))
        Y_validate_1[:,0] = 1
    elif ok_S and file[14] == 'S':
        ok_S = False
        X_validate_2 = x
        Y_validate_2 = np.zeros((x.shape[0],2))
        Y_validate_2[:,1] = 1
    else: 
        # print(x.shape)

        # create output vector. If file begins with 'P', then y = [1,0], else y = [0,1], having the same dimension as x
        if file[14] == 'P':
            y = np.zeros((x.shape[0],2))
            y[:,0] = 1
        else:
            y = np.zeros((x.shape[0],2))
            y[:,1] = 1

        X.append(x)
        Y.append(y)


Train and test the model on the 12 remaining sambples. Check for accuracies and pick the model with the best accuracy.

In [12]:
predicted_labels = []
true_labels = []

kernels, chans, samples = 1, 14, 1000

loo = LeaveOneOut()

Accuracies = []

best_acc = 0
best_model = None

for train_index, test_index in loo.split(X):
    X_train = [X[i] for i in train_index]
    Y_train = [Y[i] for i in train_index]
    X_test = [X[i] for i in test_index]
    Y_test = [Y[i] for i in test_index]

    X_train = np.concatenate(X_train)
    Y_train = np.concatenate(Y_train)
    X_test = np.concatenate(X_test)
    Y_test = np.concatenate(Y_test)

    print("ITERATION NUMBER: ", len(Accuracies) + 1)

    # convert data to NHWC (trials, channels, samples, kernels) format.
    X_train = X_train.reshape(X_train.shape[0], chans, samples, kernels)
    X_test = X_test.reshape(X_test.shape[0], chans, samples, kernels)

    model = EEGNet(nb_classes=2, Chans=chans, Samples=samples, dropoutRate=0.5, kernLength=32, F1=8, D=2, F2=16, dropoutType='Dropout')

    # compile the model and set the optimizers
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

    # count number of parameters in the model
    numParams    = model.count_params()     

    # set a valid path for the system to record model checkpoints
    # checkpointer = ModelCheckpoint(filepath='/tmp/checkpoint.h5', verbose=1, save_best_only=True)

    class_weights = {0:1, 1:1}

    fitted_model = model.fit(X_train, Y_train, batch_size = 32, epochs = 50, verbose = 2, 
                             class_weight=class_weights)
    
    # make prediction on test set.
    probs       = model.predict(X_test)
    preds       = probs.argmax(axis = -1)
    acc         = balanced_accuracy_score(Y_test.argmax(axis=-1), preds)
    print("Classification accuracy: %f " % (acc))

    #pick best predictor for class P
    if acc > best_acc and Y_test[0][0] == 1:
        best_acc = acc
        best_model = model
        print("Best model updated")

    Accuracies.append(acc)

print("Mean classification accuracy: %f " % (np.mean(Accuracies)))

ITERATION NUMBER:  1
Epoch 1/50
34/34 [==============================] - 11s 240ms/step - loss: 0.6821 - accuracy: 0.5983
Epoch 2/50
34/34 [==============================] - 8s 238ms/step - loss: 0.6048 - accuracy: 0.6760
Epoch 3/50
34/34 [==============================] - 8s 233ms/step - loss: 0.5507 - accuracy: 0.7378
Epoch 4/50
34/34 [==============================] - 8s 238ms/step - loss: 0.5480 - accuracy: 0.7219
Epoch 5/50
34/34 [==============================] - 8s 236ms/step - loss: 0.5314 - accuracy: 0.7444
Epoch 6/50
34/34 [==============================] - 8s 232ms/step - loss: 0.5173 - accuracy: 0.7519
Epoch 7/50
34/34 [==============================] - 8s 234ms/step - loss: 0.4967 - accuracy: 0.7753
Epoch 8/50
34/34 [==============================] - 8s 237ms/step - loss: 0.4508 - accuracy: 0.7968
Epoch 9/50
34/34 [==============================] - 8s 235ms/step - loss: 0.4393 - accuracy: 0.8165
Epoch 10/50
34/34 [==============================] - 8s 233ms/step - loss: 0.4

c:\Users\HTPDIR-Laptop1\eeg_net_env\Lib\site-packages\sklearn\metrics\_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


Epoch 1/50
33/33 [==============================] - 11s 238ms/step - loss: 0.6591 - accuracy: 0.6254
Epoch 2/50
33/33 [==============================] - 8s 235ms/step - loss: 0.6011 - accuracy: 0.6768
Epoch 3/50
33/33 [==============================] - 8s 234ms/step - loss: 0.5512 - accuracy: 0.7340
Epoch 4/50
33/33 [==============================] - 8s 242ms/step - loss: 0.4955 - accuracy: 0.7903
Epoch 5/50
33/33 [==============================] - 8s 246ms/step - loss: 0.3935 - accuracy: 0.8608
Epoch 6/50
33/33 [==============================] - 8s 237ms/step - loss: 0.3359 - accuracy: 0.8866
Epoch 7/50
33/33 [==============================] - 8s 252ms/step - loss: 0.2869 - accuracy: 0.9037
Epoch 8/50
33/33 [==============================] - 8s 241ms/step - loss: 0.2844 - accuracy: 0.9161
Epoch 9/50
33/33 [==============================] - 8s 236ms/step - loss: 0.2469 - accuracy: 0.9171
Epoch 10/50
33/33 [==============================] - 8s 238ms/step - loss: 0.2456 - accuracy: 0.927

c:\Users\HTPDIR-Laptop1\eeg_net_env\Lib\site-packages\sklearn\metrics\_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


Epoch 1/50
33/33 [==============================] - 11s 235ms/step - loss: 0.6758 - accuracy: 0.6095
Epoch 2/50
33/33 [==============================] - 8s 243ms/step - loss: 0.6008 - accuracy: 0.6770
Epoch 3/50
33/33 [==============================] - 8s 241ms/step - loss: 0.5698 - accuracy: 0.7126
Epoch 4/50
33/33 [==============================] - 8s 248ms/step - loss: 0.5262 - accuracy: 0.7541
Epoch 5/50
33/33 [==============================] - 8s 245ms/step - loss: 0.4806 - accuracy: 0.7946
Epoch 6/50
33/33 [==============================] - 8s 244ms/step - loss: 0.4490 - accuracy: 0.7956
Epoch 7/50
33/33 [==============================] - 8s 237ms/step - loss: 0.4025 - accuracy: 0.8428
Epoch 8/50
33/33 [==============================] - 8s 242ms/step - loss: 0.3558 - accuracy: 0.8669
Epoch 9/50
33/33 [==============================] - 8s 244ms/step - loss: 0.3006 - accuracy: 0.9074
Epoch 10/50
33/33 [==============================] - 8s 252ms/step - loss: 0.2610 - accuracy: 0.926

...

c:\Users\HTPDIR-Laptop1\eeg_net_env\Lib\site-packages\sklearn\metrics\_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


Epoch 1/50
33/33 [==============================] - 14s 285ms/step - loss: 0.6818 - accuracy: 0.5595
Epoch 2/50
33/33 [==============================] - 10s 287ms/step - loss: 0.6277 - accuracy: 0.6637
Epoch 3/50
33/33 [==============================] - 10s 306ms/step - loss: 0.5884 - accuracy: 0.6862
Epoch 4/50
33/33 [==============================] - 9s 284ms/step - loss: 0.5405 - accuracy: 0.7554
Epoch 5/50
33/33 [==============================] - 9s 269ms/step - loss: 0.4766 - accuracy: 0.8197
Epoch 6/50
33/33 [==============================] - 10s 289ms/step - loss: 0.3885 - accuracy: 0.8567
Epoch 7/50
33/33 [==============================] - 9s 271ms/step - loss: 0.3406 - accuracy: 0.8860
Epoch 8/50
33/33 [==============================] - 8s 254ms/step - loss: 0.3369 - accuracy: 0.8869
Epoch 9/50
33/33 [==============================] - 9s 261ms/step - loss: 0.2790 - accuracy: 0.9006
Epoch 10/50
33/33 [==============================] - 9s 261ms/step - loss: 0.2427 - accuracy: 0.

c:\Users\HTPDIR-Laptop1\eeg_net_env\Lib\site-packages\sklearn\metrics\_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


Epoch 1/50
32/32 [==============================] - 11s 260ms/step - loss: 0.6819 - accuracy: 0.5716
Epoch 2/50
32/32 [==============================] - 8s 255ms/step - loss: 0.6333 - accuracy: 0.6697
Epoch 3/50
32/32 [==============================] - 8s 260ms/step - loss: 0.5985 - accuracy: 0.6797
Epoch 4/50
32/32 [==============================] - 8s 265ms/step - loss: 0.5616 - accuracy: 0.7287
Epoch 5/50
32/32 [==============================] - 8s 255ms/step - loss: 0.5359 - accuracy: 0.7457
Epoch 6/50
32/32 [==============================] - 8s 259ms/step - loss: 0.4893 - accuracy: 0.7958
Epoch 7/50
32/32 [==============================] - 8s 249ms/step - loss: 0.4277 - accuracy: 0.8308
Epoch 8/50
32/32 [==============================] - 8s 251ms/step - loss: 0.3639 - accuracy: 0.8729
Epoch 9/50
32/32 [==============================] - 8s 256ms/step - loss: 0.2920 - accuracy: 0.9079
Epoch 10/50
32/32 [==============================] - 8s 249ms/step - loss: 0.2374 - accuracy: 0.944

c:\Users\HTPDIR-Laptop1\eeg_net_env\Lib\site-packages\sklearn\metrics\_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


Test the best model found previously on the validation samples

In [14]:
X_validate_1 = X_validate_1.reshape(X_validate_1.shape[0], 14, 1000, 1)
X_validate_2 = X_validate_2.reshape(X_validate_2.shape[0], 14, 1000, 1)

X_validate = np.concatenate((X_validate_1, X_validate_2))
Y_validate = np.concatenate((Y_validate_1, Y_validate_2))

X_validate = X_validate.reshape(X_validate.shape[0], 14, 1000, 1)


probs_1 = best_model.predict(X_validate_1)
preds_1 = probs_1.argmax(axis = -1)
acc_1 = balanced_accuracy_score(Y_validate_1.argmax(axis=-1), preds_1)



probs_2 = best_model.predict(X_validate_2)
preds_2 = probs_2.argmax(axis = -1)
acc_2 = balanced_accuracy_score(Y_validate_2.argmax(axis=-1), preds_2)

probs = best_model.predict(X_validate)
preds = probs.argmax(axis = -1)
acc = balanced_accuracy_score(Y_validate.argmax(axis=-1), preds)

print("Classification accuracy for P: %f " % (acc_1))
print("Classification accuracy for S: %f " % (acc_2))
print("Classification accuracy for both: %f " % (acc))

3/3 [==============================] - 0s 49ms/step


c:\Users\HTPDIR-Laptop1\eeg_net_env\Lib\site-packages\sklearn\metrics\_classification.py:2184: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


6/6 [==============================] - 0s 49ms/step
Classification accuracy for P: 1.000000 
Classification accuracy for S: 0.444444 
Classification accuracy for both: 0.722222 
